## Optimizing `frozen_epochs`, `unfrozen_epochs`, `base_lower_bound`, `factors`, `ratio_split_lower_bound` , parameter of `FastAIScreenDetector`

In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 5.4 s, sys: 544 ms, total: 5.95 s
Wall time: 5.34 s


In [2]:
resize_factor = [2]
frozen_epochs = [4, 7]
unfrozen_epochs = [4, 7]
base_lower_bound = [5, 7, 10, 15]
erosion_dilation_kernel_size = [20, 50, 80, 150]
ratio_split_lower_bound = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

In [3]:
# resize_factor = [8]
# frozen_epochs = [1]
# unfrozen_epochs = [0]
# base_lower_bound = [5]
# erosion_dilation_kernel_size = [80]
# ratio_split_lower_bound = [0.5]

In [4]:
train = list(product(resize_factor, frozen_epochs, unfrozen_epochs))
train_names = ['resize_factor', 'frozen_epochs', 'unfrozen_epochs']

post_processing = list(product(base_lower_bound, erosion_dilation_kernel_size, ratio_split_lower_bound))
post_processing_names = ['base_lower_bound', 'erosion_dilation_kernel_size', 'ratio_split_lower_bound']

all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}

detector = FastAIScreenDetector()
actual_detector = AnnotatedSampledVideoScreenDetector()

In [5]:
def filtered_by(name, used):
    return any([lecture in str(name) for lecture in used]) and 'frame002000' in str(name)

def split_by(name, validation):
    return any([lecture in str(name) for lecture in validation])

### Cross validation splits by lecture

In [6]:
def make_splits(all_lectures):
    Split = namedtuple('Split', ['train', 'valid'])
    kf = KFold(n_splits=5, shuffle=True, random_state=123)
    splits = {}
    for j, split in enumerate(kf.split(all_lectures)):    
        train_lectures = [all_lectures[index] for index in split[0]]
        valid_lectures = [all_lectures[index] for index in split[1]]
        valid_frames = [frame for lecture in valid_lectures for frame in all_frames_grouped_by_videos[lecture]]
        splits[j] = Split(train=train_lectures, valid=valid_lectures)
    return splits

In [7]:
def model_selection(all_lectures, train_names, post_processing_names, default_filtered_by, default_split_by):
    splits = make_splits(all_lectures)
    df_all = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count', 'kfold_split'])

    for train_values in tqdm(train):
        resize_factor, frozen_epochs, unfrozen_epochs = train_values
        CONFIGURATION['resize_factor'] = str(resize_factor)
        CONFIGURATION['frozen_epochs'] = str(frozen_epochs)
        CONFIGURATION['unfrozen_epochs'] = str(unfrozen_epochs)

        for j in splits.keys():
            filtered_by = partial(default_filtered_by, used=splits[j].train + splits[j].valid)
            split_by = partial(default_split_by, validation=splits[j].valid)

            detector = FastAIScreenDetector(filtered_by=filtered_by, valid_func=split_by)
            detector.train()

            valid_frames = [frame for frame in all_frames if split_by(frame.pathname)]
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)

            for post_processing_values in post_processing:    
                preds = detector.post_processing_batch(sem_preds, valid_frames)
                wrong_count, ious, _ = evaluate(actuals, preds)

                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_values + post_processing_values + (iou_score, wrong_count, j)
                df_all.to_csv("model_selection.csv")
            df_all.to_csv("model_selection.csv")
    return df_all

In [8]:
%%time
df_all = model_selection(all_lectures, train_names, post_processing_names, filtered_by, split_by)

  0%|          | 0/4 [00:00<?, ?it/s]

epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.778720,0.534059,0.691155,0.000904,0.000452,00:11
1,0.687508,3.176260,0.686907,0.000000,0.000000,00:01
2,1.551267,8.060762,0.311265,0.470666,0.307759,00:01
3,4.459927,1.876106,0.336627,0.480036,0.315822,00:01


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,4.889421,1.632936,0.343991,0.482828,0.318245,00:01
1,4.573780,0.322519,0.889670,0.862568,0.778253,00:01
2,3.363383,0.235194,0.970129,0.949487,0.904666,00:01
3,2.599056,0.239909,0.956255,0.922737,0.858369,00:01


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.974867,0.567435,0.760921,0.026712,0.013573,00:03
1,0.851346,21.190159,0.289969,0.391743,0.247177,00:01
2,5.482977,27.444206,0.766921,0.000000,0.000000,00:01
3,15.167083,10.384185,0.766921,0.000000,0.000000,00:01


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,21.083714,9.358636,0.766921,0.000000,0.000000,00:01
1,20.570957,2.148708,0.766921,0.000000,0.000000,00:01
2,15.328886,0.603159,0.766898,0.000053,0.000026,00:01
3,11.753401,0.375370,0.770042,0.032925,0.016917,00:01


...

epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.847624,0.559730,0.715818,0.015317,0.007727,00:01
1,0.639610,1.492060,0.717438,0.000000,0.000000,00:01
2,0.737250,10.392610,0.286405,0.441080,0.283515,00:01
3,4.334046,0.199564,0.987780,0.978152,0.957257,00:01
4,3.483696,0.229033,0.935923,0.871913,0.772989,00:01
5,2.931106,0.223450,0.936806,0.872930,0.774812,00:01
6,2.557880,0.213314,0.949947,0.902186,0.821944,00:01


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.319061,0.203227,0.959727,0.923177,0.857350,00:01
1,0.343644,0.172836,0.984351,0.971694,0.944951,00:01
2,0.325196,0.139969,0.986970,0.976529,0.954142,00:01
3,0.289492,0.131988,0.970221,0.941213,0.890275,00:01
4,0.306719,0.093667,0.986497,0.975665,0.952488,00:01
5,0.278646,0.081207,0.989178,0.980944,0.962607,00:01
6,0.254938,0.077594,0.989352,0.981271,0.963234,00:01


100%|██████████| 4/4 [17:34<00:00, 263.50s/it]

CPU times: user 18min 50s, sys: 1min 35s, total: 20min 26s
Wall time: 17min 34s


,resize_factor,frozen_epochs,unfrozen_epochs,base_lower_bound,erosion_dilation_kernel_size,ratio_split_lower_bound,iou,wrong_count,kfold_split
0,2.0,4.0,4.0,5.0,20.0,0.3,0.737472,68.0,0.0
1,2.0,4.0,4.0,5.0,20.0,0.4,0.737472,68.0,0.0
2,2.0,4.0,4.0,5.0,20.0,0.5,0.737472,68.0,0.0
3,2.0,4.0,4.0,5.0,20.0,0.7,0.737472,68.0,0.0
4,2.0,4.0,4.0,5.0,20.0,0.8,0.737472,68.0,0.0
...,...,...,...,...,...,...,...,...,...
1915,2.0,7.0,7.0,15.0,150.0,0.4,0.943484,25.0,4.0
1916,2.0,7.0,7.0,15.0,150.0,0.5,0.943484,25.0,4.0
1917,2.0,7.0,7.0,15.0,150.0,0.7,0.943484,25.0,4.0
1918,2.0,7.0,7.0,15.0,150.0,0.8,0.943484,25.0,4.0
